# Mount drive and append path to PYTONPATH


In [ ]:
from google import colab
import os
import sys

colab.drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/DeepLCMS/gpu_modules")

# Import and install libraries

In [ ]:
%%capture
!pip install lightning
!pip install timm
!pip install torchinfo
!pip install scikit-posthocs
!pip install optuna
!pip install torchcam

In [ ]:
import colab_functions
import colab_utils
import pandas as pd
import prepare_data
import timm
import train_NN
from lightning.pytorch import loggers, callbacks, tuner, trainer
from pathlib import Path

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Unzip data

In [ ]:
!unzip -q "*.zip"

# Check if GPU is used

In [ ]:
device = colab_functions.get_device()

# Taking a look at the list of Timm pretrained models

In [ ]:
timm_model_db = pd.read_csv(
    "https://raw.githubusercontent.com/huggingface/pytorch-image-models/main/results/results-imagenet.csv"
).assign(param_count=lambda df: df.param_count.str.replace(",", "").astype(float))

In [ ]:
timm_family_list = [
    "Aggregating Nested Transformers",
    "BiT",
    "Bottleneck Transformers",
    "CaiT",
    "CoaT",
    "CoAtNet",
    "ConvNeXt",
    "ConvNeXt-V2",
    "ConViT",
    "CspNet",
    "DeiT",
    "DeiT-III",
    "DenseNet",
    "DLA",
    "DPN",
    "EdgeNeXt",
    "EfficientFormer",
    "EfficientNet",
    "FBNet",
    "MixNet",
    "MNASNet",
    "MobileNet",
    "Single-Path NAS",
    "TinyNet",
    "EfficientViT",
    "EVA",
    "FastViT",
    "FlexiViT",
    "FocalNet",
    "GCViT",
    "GhostNet",
    "GhostNet-V2",
    "gMLP",
    "HRNet",
    "InceptionNeXt",
    "Inception-V3",
    "Inception-ResNet-V2",
    "Lambda",
    "LeViT",
    "MaxViT",
    "MobileNet-V3",
    "LCNet",
    "MobileOne",
    "MobileViT",
    "MobileViT-V2",
    "MViT-V2",
    "NASNet-A",
    "NesT",
    "NFNet-F",
    "NF-RegNet",
    "PNasNet",
    "RegNet",
    "RegNetZ",
    "RepVGG",
    "RepGhostNet",
    "RepViT",
    "ResMLP",
    "ResNeXt",
    "SEResNet",
    "ResNet-RS",
    "Res2Net",
    "ResNeSt",
    "ReXNet",
    "Swin",
    "TResNet",
    "Twins",
    "Xception",
    "XCiT",
]
len(timm_family_list)

In [ ]:
cleaned_timm_model_list = pd.Series(timm_family_list).str.replace(" ", "").str.lower()

In [ ]:
models_by_family = []

for model_name in cleaned_timm_model_list:
    query_result = (
        timm_model_db.query("model.str.contains(@model_name) and 10 < param_count < 20")
        .sort_values(by="param_count")
        .head(1)
    )

    if query_result.shape[0] != 0:
        result_dict = {
            "model_family": query_result.model.values[0],
            "minimal_param_model_count": query_result.param_count.values[0],
        }

        models_by_family.append(result_dict)

models_by_family_df = pd.DataFrame(models_by_family)

models_by_family_df.sort_values(by="model_family")

# Inspect a model and its dataloader


In [ ]:
PRETRAINED_MODEL = "convnextv2_nano.fcmae_ft_in22k_in1k_384"

model = train_NN.PretrainedModel(
    pretrained_model_name=PRETRAINED_MODEL, learning_rate=0.001
)
datamodule = prepare_data.LCMSDataModule(
    model,
    data_dir=Path("/content/ST001618_Opium_study_LC_MS_500"),
)
model.show_architecture()

In [ ]:
datamodule.inspect_dataloader("train")

# Training loop

In [ ]:
for model_name in models_by_family_df.model_family:
    try:
        model = train_NN.PretrainedModel(
            pretrained_model_name=model_name, learning_rate=0.001
        )
        datamodule = prepare_data.LCMSDataModule(
            model,
            data_dir=Path("/content/ST001618_Opium_study_LC_MS_500"),
        )

        logger = loggers.CSVLogger("logs", name=str(model_name))

        trainer_ = trainer.Trainer(
            max_epochs=50,
            log_every_n_steps=1,
            logger=logger,
            precision="16-mixed",
            callbacks=[
                callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=10)
            ],
        )
        """
    Note: tried using auto_lr tuner before each model training
    but sometimes got very high lr values, such as 0.5, so
    decided not to use this feature.

    In case needed, here is the implementation (no need to
    change lr manually, this autoupdates it):

    # # Create a Tuner
    # tuner_ = tuner.Tuner(trainer_)

    # # Find optimal learning rate
    # lr_finder = tuner_.lr_find(model,
    #                            datamodule = datamodule,
    #                            early_stop_threshold=None)

    # # Extract best learning rate
    # best_lr = lr_finder.suggestion()

    """
        # Train the model using the found learning rate
        trainer_.fit(model=model, datamodule=datamodule)

        del model, datamodule, trainer_

    except RuntimeError as e:
        print(f"{model_name} could not run because {e}")

In [ ]:
results_df = colab_functions.get_experiment_results()
results_df.to_csv("pretrained_model_results.csv", index=False)
colab_functions.plot_experiment_results(results_df)

In [ ]:
df = pd.read_csv("pretrained_model_results.csv")

In [ ]:
colab_functions.plot_experiment_results(df, bbox_to_anchor=(1.2, 0.97))

In [ ]:
# Filter rows with variable containing 'val'
filtered_df = df[df["variable"].str.contains("val")]


# Apply aggregation with groupby and calculate max and min values per metric
result_df_max = (
    filtered_df.groupby(["variable", "experiment"])
    .value.max()
    .to_frame()
    .reset_index(drop=False)
    .query('variable != "val_loss"')
    .pivot(index="experiment", columns="variable", values="value")
    .round(3)
)

result_df_min = (
    filtered_df.groupby(["variable", "experiment"])
    .value.min()
    .to_frame()
    .reset_index(drop=False)
    .query('variable == "val_loss"')
    .pivot(index="experiment", columns="variable", values="value")
    .round(3)
)

# concatanate the two dfs
result_df = pd.concat([result_df_max, result_df_min], axis="columns").sort_values(
    by="val_loss"
)

In [ ]:
result_df.merge(
    models_by_family_df, left_index=True, right_on="model_family"
).set_index("model_family")

# Candidates for next experiment

In [ ]:
top_5_model_families = ["mobileone", "densenet", "mobilevitv2", "convnext", "repvit"]

top_models_by_family = []
for model_family in top_5_model_families:
    query_result = timm_model_db.query("model.str.contains(@model_family)")
    top_models_by_family.append(query_result)

pd.concat(top_models_by_family)